In [12]:
import sqlite3

# Conectando ao SQLite
conn = sqlite3.connect('pokedex.db')
cursor = conn.cursor()

# Criando a tabela
cursor.execute('''
CREATE TABLE IF NOT EXISTS pokemons (
    id INTEGER PRIMARY KEY,
    name TEXT,
    url TEXT,
    abilities TEXT,
    base_experience INTEGER,
    forms TEXT,
    game_indices TEXT,
    height INTEGER,
    held_items TEXT,
    is_default BOOLEAN,
    location_area_encounters TEXT,
    moves TEXT,
    n_order INTEGER,
    past_types TEXT,
    species TEXT,
    sprites TEXT,
    stats TEXT,
    types TEXT,
    weight INTEGER
);
''')
conn.commit()

print("Tabela 'pokemons' criada com sucesso!")
cursor.close()
conn.close()


Tabela 'pokemons' criada com sucesso!


In [13]:
import sqlite3

# Conectando ao SQLite
conn = sqlite3.connect('pokedex.db')
cursor = conn.cursor()

# Deletando todos os registros da tabela 'pokemons'
cursor.execute('DELETE FROM pokemons')
conn.commit()

print("Todos os dados da tabela 'pokemons' foram deletados!")
cursor.close()
conn.close()


Todos os dados da tabela 'pokemons' foram deletados!


In [14]:
import pika
import sqlite3
import json

# Configuração para conectar ao RabbitMQ
credentials = pika.PlainCredentials('user', 'password')
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost', 5672, '/', credentials))
channel = connection.channel()

# Garantindo que a fila exista
channel.queue_declare(queue='pokemon_queue', durable=True)

# Conectando ao SQLite
conn = sqlite3.connect('pokedex.db')
cursor = conn.cursor()

# Função callback para processar mensagens da fila
def callback(ch, method, properties, body):
    # Convertendo a mensagem de volta em um dicionário
    data = json.loads(body)

    # Verificação para 'n_order' e 'order'
    n_order = data.get('n_order', data.get('n_order', None))
    if n_order is None:
        print(f"Erro: 'n_order' ou 'order' não encontrados na mensagem para o Pokémon {data['name']}. Ignorando...")
        return

    # Inserindo dados no SQLite
    cursor.execute('''
    INSERT INTO pokemons (
        name, url, abilities, base_experience, forms, game_indices,
        height, held_items, id, is_default, location_area_encounters,
        moves, n_order, past_types, species, sprites, stats, types, weight
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        data['name'], data['url'], json.dumps(data['abilities']), data['base_experience'],
        json.dumps(data['forms']), json.dumps(data['game_indices']), data['height'],
        json.dumps(data['held_items']), data['id'], data['is_default'],
        data['location_area_encounters'], json.dumps(data['moves']), n_order,
        json.dumps(data['past_types']), json.dumps(data['species']), json.dumps(data['sprites']),
        json.dumps(data['stats']), json.dumps(data['types']), data['weight']
    ))
    conn.commit()

    print(f"Dados do Pokémon {data['name']} foram inseridos no SQLite.")

# Informar ao RabbitMQ que queremos começar a consumir mensagens da fila
channel.basic_consume(queue='pokemon_queue',
                      on_message_callback=callback,
                      auto_ack=True)  # Automaticamente confirma o recebimento das mensagens

print("Esperando mensagens da fila. Para sair pressione CTRL+C")
channel.start_consuming()

# Não se esqueça de fechar as conexões ao terminar
cursor.close()
conn.close()
connection.close()


Esperando mensagens da fila. Para sair pressione CTRL+C
Dados do Pokémon bulbasaur foram inseridos no SQLite.
Dados do Pokémon ivysaur foram inseridos no SQLite.


KeyboardInterrupt: 